In [ ]:
import pickle
import pandas as pd
from tqdm import tqdm

from math import ceil

import os

In [ ]:
output_dir="../../data/paragraph_pos"
pkl_files = [fname for fname in os.listdir(output_dir) if fname.startswith("train_paragraph_pos_") and fname.endswith(".pkl")]

In [ ]:
pkl_files = sorted([fname for fname in os.listdir(output_dir) if fname.startswith("paragraph_pos_") and fname.endswith(".pkl")])
print(pkl_files)
dfs = []
for fname in tqdm(pkl_files, desc="paragraph_pos_idx.pkl 병합: "):
    df = pd.read_pickle(os.path.join(output_dir, fname))
    dfs.append(df)
all_df = pd.concat(dfs, ignore_index=True)

In [ ]:
train_paragraph_id = pd.read_csv("../../data/train_paragraph_id.csv")

In [ ]:
# (title, paragraph_index) <-> paragraph_pos 매핑
key_to_pos = {
    (row['title'], row['paragraph_index']): row['paragraph_pos']
    for _, row in tqdm(all_df.iterrows(), total=len(all_df))
}
tqdm.pandas(desc="paragraph_pos 매핑")
train_paragraph_id['paragraph_pos'] = train_paragraph_id.progress_apply(
    lambda row: key_to_pos.get((row['title'], row['paragraph_index']), None), axis=1
)
final_df = train_paragraph_id.sort_values('ID')[['ID', 'title', 'paragraph_index',
                                                 'paragraph_pos', 'generated']]


In [ ]:
len(final_df)

In [ ]:
final_df.head(10)

In [ ]:
titles = final_df['title'].unique()
n_files = 6
titles_per_file = ceil(len(titles) / n_files)
title_groups = [titles[i*titles_per_file:(i+1)*titles_per_file] for i in range(n_files)]

output_dir = "../../data/train_id_paragraph_pos"
os.makedirs(output_dir, exist_ok=True)

for i, title_group in tqdm(enumerate(title_groups), total=n_files):
    chunk = final_df[final_df['title'].isin(title_group)]
    chunk = chunk.sort_values('ID')
    chunk.to_pickle(os.path.join(output_dir, f"train_paragraph_pos_{i}.pkl"))
    print(f"Saved: train_paragraph_pos_{i}.pkl with {len(chunk)} rows, titles: {len(title_group)}")